# Ringland Festival kaart - GPX data

In [1]:
import pandas as pd
import geopandas
from shapely.geometry import Point

In [2]:
df = pd.read_csv('../01-data/processed/curieuzeneuzen-results.csv')

In [3]:
df.head()

,SetId,mean,span,Lat,Lon
0,1,54.637056,0.888894,51.193319,4.463377
1,2,38.844720,3.632833,51.205845,4.407636
2,3,17.600972,0.930665,51.220553,4.413698
3,4,45.431083,0.342342,51.213435,4.393113
4,5,29.334421,7.797653,51.201084,4.446258


In [4]:
gdf = geopandas.GeoDataFrame(df[['mean']], geometry=df[['Lon', 'Lat']].apply(lambda x: Point(x['Lon'], x['Lat']), axis=1))

In [5]:
gdf.head()

,mean,geometry
0,54.637056,POINT (4.46337694 51.19331943)
1,38.844720,POINT (4.40763586 51.20584495)
2,17.600972,POINT (4.4136976 51.22055294)
3,45.431083,POINT (4.3931128 51.2134349)
4,29.334421,POINT (4.44625811 51.20108353)


In [6]:
def to_gpx(gdf, fpath=None):
    """
    Only for Points for now
    """
    
    import gpxpy.gpx
    
    geoms = gdf.geometry
    
    gpx = gpxpy.gpx.GPX()
    wps = []
    
    for p in geoms:
        wps.append(gpxpy.gpx.GPXWaypoint(latitude=p.y, longitude=p.x))
    
    gpx.waypoints.extend(wps)
    
    if fpath is None:
        return gpx.to_xml()
    else:
        with open(fpath, 'w') as f:
            f.write(gpx.to_xml())

    lager dan 25: donker groen
    25-30: licht groen
    31-35: geel
    36-40: licht oranje
    41-45: midden oranje
    46-50: donker oranje
    51-55: licht rood
    56-60: midden rood
    61-65: donker rood
    hoger dan 65: zwart

In [22]:
check = []

In [23]:
temp = gdf[gdf['mean'] < 25]
to_gpx(temp, 'curieuzeneuzen-25.gpx')
check.append(len(temp))

In [24]:
for i, j in zip(range(25,65,5), range(30,70,5)):
    temp = gdf[(gdf['mean'] >= i) & (gdf['mean'] < j)]
    to_gpx(temp, 'curieuzeneuzen-{0}-{1}.gpx'.format(i, j))
    check.append(len(temp))

In [25]:
temp = gdf[gdf['mean'] >= 65]
to_gpx(temp, 'curieuzeneuzen-65.gpx')
check.append(len(temp))

In [28]:
assert sum(check) == len(gdf['mean'])